# LangQA – Language-powered question and answer system

## Imports

In [11]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import pipeline, TrainingArguments
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
import warnings

warnings.filterwarnings('ignore')

## Load Data

https://huggingface.co/datasets/nlpie/Llama2-MedTuned-Instructions

In [5]:
# Load dataset
dataset = load_dataset('nlpie/Llama2-MedTuned-Instructions')

Generating validation split: 100%|██████████| 70066/70066 [00:00<00:00, 1771353.08 examples/s]


In [6]:
train_data = dataset['train'].select(indices=range(1000))

train_data

Dataset({
    features: ['instruction', 'input', 'output', 'source'],
    num_rows: 1000
})

In [7]:
# Selecting the lines to test the model
test_data = dataset['train'].select(indices=range(1000, 1200))

## Understanding the format of the text

In [8]:
for i in range(3):
    data = dataset['train'][i]
    print(f"Data point {i + 1}:")
    print("Instruction:", data['instruction'])
    print("Input:", data['input'])
    print("Output:", data['output'])
    print("\n-----------------------------\n")

Data point 1:
Instruction: In your role as a medical professional, address the user's medical questions and concerns.
Input: My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.
Output: Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.

-----------------------------



## Automating the Creation of Prompts for Model Training

In [9]:
# Defines a function that takes a dictionary named sample
def create_prompt(sample):

    # Defines a pre_prompt string that serves as a template for the first part of the prompt
    pre_prompt = """[INST]<<SYS>> {instruction}\n"""

    # Concatenates pre_prompt with additional strings to form the complete prompt
    prompt = pre_prompt + "{input}" +"[/INST]"+"\n{output}"

    # Assigns the value of the 'instruction' key of the dictionary sample to the variable example_instruction
    example_instruction = sample['instruction']

    # Assigns the value of the 'input' key of the dictionary sample to the variable example_input
    example_input = sample['input']

    # Assigns the value of the 'output' key of the dictionary sample to the variable example_output
    example_output = sample['output']

    # Creates an instance of PromptTemplate with the previously defined prompt and input variables
    prompt_template = PromptTemplate(template = prompt,
    input_variables = ["instruction", "input", "output"])

    # Uses the format method of the prompt_template instance to replace the variables
    # in the template with the specified values
    prompt_unico = prompt_template.format(instruction = example_instruction,
                                          input = example_input,
                                          output = example_output)

    # Returns the formatted prompt
    return prompt_unico

In [10]:
# Testing the function
prompt = create_prompt(train_data[0])
print(prompt)

[INST]<<SYS>> In your role as a medical professional, address the user's medical questions and concerns.
My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.[/INST]
Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.


## Quantization Process

In [12]:
# Enables loading of the base model with 4-bit precision
use_4bit = True

# Sets the dtype for the base model
bnb_4bit_compute_dtype = "float16"

# Quantization type
bnb_4bit_quant_type = "nf4"

# Disables double quantization
use_nested_quant = False

# Sets the dtype for computation in PyTorch
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [13]:
# Defining the config
bnb_config = BitsAndBytesConfig(load_in_4bit = use_4bit,
                                bnb_4bit_quant_type = bnb_4bit_quant_type,
                                bnb_4bit_compute_dtype = compute_dtype,
                                bnb_4bit_use_double_quant = use_nested_quant)

In [14]:
# Verifying if the GPU supports bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("The GPU suporrts bfloat16. You can accelerate the train using bf16=True")
        print("=" * 80)

The GPU suporrts bfloat16. You can accelerate the train using bf16=True


## Load the LLM and the Tokenizer

https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [17]:
# LLM
llm_name = "NousResearch/Llama-2-7b-chat-hf"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name)

# Load the base model with quantization
modelo = AutoModelForCausalLM.from_pretrained(llm_name,
                                              quantization_config = bnb_config,
                                              device_map = "auto",
                                              use_cache = False)

Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.21s/it]


In [18]:
# Use the EOS token from the tokenizer to pad at the end of each sequence
tokenizer.pad_token = tokenizer.eos_token

# Enable padding at the end of each sentence
tokenizer.padding_side = "right"